# Extractive Text Summarization

- Split the document into sentences (sentence tokenization)
- Assign a score to each sentence
- Pick the top N sentences based on the scores

- Score = Average (non-zero TF-IDF of words in the sentence) 
    - Unimportant words have smaller values in the TF-IDF matrix
    - Important words appearing more often in the sentence will have an even higher score.
    - Average instead of just summing to avoid bias towards longer sentences. 
    - Non-zero elements because the TF-IDF is very sparse and don't want to choose based on variety of words

- TextRank score


In [223]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [224]:
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/amarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/amarov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/amarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [225]:
df = pd.read_csv("https://github.com/febse/data/raw/refs/heads/main/ta/BBC%20News%20Train.csv.zip")
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [226]:
doc = df.iloc[12]

print(textwrap.fill(doc["Text"], replace_whitespace=False, fix_sentence_endings=True))

uk coal plunges into deeper loss shares in uk coal have fallen after
the mining group reported losses had deepened to £51.6m in 2004 from
£1.2m.  the uk s biggest coal producer blamed geological problems
industrial action and  operating flaws  at its deep mines for its
worsening fortunes.  the south yorkshire company  led by new chief
executive gerry spindler  said it hoped to return to profit in 2006.
in early trade on thursday  its shares were down 10% at 119 pence.  uk
coal said it was making  significant progress  in shaking up the
business.  it had introduced new wage structures  a new daily
maintenance regime for machinery at its mines and methods to continue
mining in adverse conditions.  the company said these actions should
significantly uplift earnings . it expected 2005 to be a  transitional
year  and to return to profitability in 2006.  the recent rise in coal
prices has failed to benefit the company as most of its output had
already been sold  it said.  total production co

In [227]:
type(doc["Text"])

str

In [228]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(doc["Text"])
sents

['uk coal plunges into deeper loss shares in uk coal have fallen after the mining group reported losses had deepened to £51.6m in 2004 from £1.2m.',
 'the uk s biggest coal producer blamed geological problems  industrial action and  operating flaws  at its deep mines for its worsening fortunes.',
 'the south yorkshire company  led by new chief executive gerry spindler  said it hoped to return to profit in 2006. in early trade on thursday  its shares were down 10% at 119 pence.',
 'uk coal said it was making  significant progress  in shaking up the business.',
 'it had introduced new wage structures  a new daily maintenance regime for machinery at its mines and methods to continue mining in adverse conditions.',
 'the company said these actions should  significantly uplift earnings .',
 'it expected 2005 to be a  transitional year  and to return to profitability in 2006.  the recent rise in coal prices has failed to benefit the company as most of its output had already been sold  it sai

In [229]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))

X = vectorizer.fit_transform(sents)

In [230]:
X.shape

(11, 113)

In [231]:
X = X.toarray()
scores = np.array([np.mean(row[row != 0]) for row in X])
print("Row-wise Mean of Non-zero Elements:\n", scores)

Row-wise Mean of Non-zero Elements:
 [0.24924015 0.25482022 0.21658328 0.34347024 0.26179554 0.40054402
 0.23270168 0.25594703 0.4472136  0.31275452 0.24686998]


In [232]:
sort_idx = np.argsort(-scores)

In [233]:
print("Top sentences:\n")

for i in sort_idx[:5]:
  print(f"%.2f: %s" % (scores[i], sents[i]))

Top sentences:

0.45: we have a long journey ahead to fix these issues.
0.40: the company said these actions should  significantly uplift earnings .
0.34: uk coal said it was making  significant progress  in shaking up the business.
0.31: we continue to make progress and great strides have already been made   said mr spindler.
0.26: it had introduced new wage structures  a new daily maintenance regime for machinery at its mines and methods to continue mining in adverse conditions.


## TextRank

TextRank is an unsupervised keyword and sentence extraction algorithm that is inspired by PageRank, the Google page ranking algorithm in the 1990s and 2000s. 

Before we can see how to apply TextRank to text summarization, we need to understand how PageRank works.
PageRank is a link analysis algorithm that assigns a numerical weighting to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of "measuring" its relative importance within the set.


```{mermaid}
graph TD
    A[Page A] -->|Link| B[Page B]
    A -->|Link| C[Page C]
    B -->|Link| D[Page D]
    C -->|Link| D
    D -->|Link| A
    D -->|Link| E[Page E]
    E -->|Link| B
```


Let's take a walk through the web. We start at a random webpage and follow a random link on that page. On the next page, we again follow a randomly chosen link. We keep doing this for a long time. We can ask a question: what is the probability that we end up on a certain page?

We can view this walk as a Markov chain. Say that all pages are $n$ and let's assume that we can reach any page from any other page (though not 
with the same probability).

In [234]:
# print()

In [235]:
import nltk
from nltk.corpus import gutenberg

nltk.download('gutenberg')


[nltk_data] Downloading package gutenberg to /home/amarov/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [236]:

alice = gutenberg.raw(fileids="carroll-alice.txt")

print(f"Text length: {len(alice)} characters")

# Remove non-word characters and spaces, lowercase

import re
alice_clean = re.sub(r'\s+', '', re.sub(r'[^\w\s]', '', alice)).lower()

print(f"Cleaned text length: {len(alice_clean)} characters")

# Convert to a sequence of consonants and vowels

alice_cv = re.sub(r'[aeiou]', 'V', re.sub(r'[^aeiou\s]', 'C', alice_clean))
alice_cv[:100]

Text length: 144395 characters
Cleaned text length: 107693 characters


'VCVCVCVCCVCCVCVCVCCVCCVCCVCCCCCVCVCCVCCVCCCCCCCCVCCVCVCVCCCCVCVCCVCCVCVVCVCVCVCCVCVCCVCCCVCVCCVCCCVC'

In [237]:
# Count the frequencies of vowels and consonants

from collections import Counter
cv_counts = Counter(alice_cv)
cv_counts

Counter({'C': 66212, 'V': 41481})

In [238]:
P_V = cv_counts['V'] / (cv_counts['V'] + cv_counts['C'])  # P(V)
P_C = cv_counts['C'] / (cv_counts['V'] + cv_counts['C'])  # P(C)

print(f"P(V): {P_V:.4f}, P(C): {P_C:.4f}")
# Count the frequencies of VV, CC, VC, CV

P(V): 0.3852, P(C): 0.6148


In [239]:
# Count the frequencies of VV, CC, VC, CV

cv_bigrams = [alice_cv[i:i+2] for i in range(len(alice_cv)-1)]
bigram_counts = Counter(cv_bigrams)
bigram_counts

Counter({'VC': 34838, 'CV': 34837, 'CC': 31374, 'VV': 6643})

In [240]:
V_to_C = 34838 / (34838 + 6643)  # P(V->V | V)
V_to_V = 6643 / (34838 + 6643)   # P(V->C | V)

C_to_C = 31374 / (31374 + 34837) # P(C->C | C)
C_to_V = 34837 / (31374 + 34837) # P(C->V | C)

print(f"P(V->V|V): {V_to_V:.2}, P(V->C|V): {V_to_C:.2}")
print(f"P(C->C|C): {C_to_C:.2}, P(C->V|C): {C_to_V:.2}")

P(V->V|V): 0.16, P(V->C|V): 0.84
P(C->C|C): 0.47, P(C->V|C): 0.53


In [241]:
# Count the frequencies of vowels and consonants

Let's now track the states of the Markov chain as we jump between vowels and consonants in "Alice's Adventures in Wonderland".

We perform a series of transitions between two states: Vowel (V) and Consonant (C). Let's denote the state we are in at step $t$ as $S_t$.
So $S_t = V$ means that at step $t$ we are at a vowel, and $S_t = C$ means we are at a consonant.

- We start by picking a state at random, say with equal probability:
  - $P(S_0 = V) = 0.5$
  - $P(S_0 = C) = 0.5$
- Next, we jump to the next state based on the transition probabilities:
  - $P(S_{t+1} = V | S_t = V) = 0.84$ (probability of going from vowel to vowel)
  - $P(S_{t+1} = C | S_t = V) = 0.16$ (probability of going from vowel to consonant)
  - $P(S_{t+1} = C | S_t = C) = 0.47$ (probability of going from consonant to consonant)
  - $P(S_{t+1} = V | S_t = C) = 0.53$ (probability of going from consonant to vowel)
  

What are the probabilities of landing at a vowel/consonant in step 1? We can use the law of total probability:

$$
\begin{align*}
P(S_1 = v) & = P(S_1 = v | S_0 = v) P(S_0 = v) + P(S_1 = v | S_0 = c) P(S_0 = c) \\
P(S_1 = c) & = P(S_1 = c | S_0 = v) P(S_0 = v) + P(S_1 = c | S_0 = c) P(S_0 = c)
\end{align*}
$$

For the following it will be convenient to express the probabilities as a vector and the transition probabilities as a matrix:

$$
\mathbf{p_t} = \begin{pmatrix} P(S_t = v) & P(S_t = c) \end{pmatrix} , \quad \mathbf{T} = \begin{pmatrix} P(S_{t+1} = v | S_t = v) & P(S_{t+1} = c | S_t = v) \\ P(S_{t+1} = v | S_t = c) & P(S_{t+1} = c | S_t = c) \end{pmatrix}
$$

Now we can express the probabilities for the two states at step $t+1$ as:

$$
\mathbf{p_{t+1}} = \mathbf{p_t} \mathbf{T}
$$

We can recursively apply this to get the probabilities at step $t$ starting from the initial probabilities $\mathbf{p_0}$:

$$
\begin{align*}
\mathbf{p_t} & = \mathbf{p_{t - 1}} \mathbf{T} \mathbf{T} \\
& = \mathbf{p_{t - 2}} \mathbf{T} \mathbf{T} \mathbf{T} \\
& \ \, \vdots \\
              & = \mathbf{p_0} \mathbf{T}^t
\end{align*}
$$

But is there a steady state distribution $\mathbf{p^*}$ such that:

$$
\mathbf{p^*} = \mathbf{p^*} \mathbf{T}
$$

The answer is provided by the Perron-Frobenius theorem, which states that an ergodic Markov chain has a unique stationary distribution. To be ergodic, the Markov chain must be irreducible (it is possible to get to any state from any state) and aperiodic (the system does not get trapped in cycles).

How do we compute $\mathbf{p^*}$?

The last equation can be rewritten as:

$$
\mathbf{p^*} (\mathbf{T} - \mathbf{I}) = 0
$$

which is an eigenvalue problem. The stationary distribution $\mathbf{p^*}$ is the left eigenvector of the transition matrix $\mathbf{T}$ corresponding to the eigenvalue 1.



In [242]:
import numpy as np

p0 = np.array([0.5, 0.5])

T = np.array([[V_to_V, V_to_C],
              [C_to_V, C_to_C]])

print("Transition Matrix:\n", T)

def markov_step(p, T):
    return np.dot(p, T)

for step in range(3):
    p0 = markov_step(p0, T)
    print(f"After step {step+1}: P(V)={p0[0]:.4f}, P(C)={p0[1]:.4f}")

Transition Matrix:
 [[0.16014561 0.83985439]
 [0.52615124 0.47384876]]
After step 1: P(V)=0.3431, P(C)=0.6569
After step 2: P(V)=0.4006, P(C)=0.5994
After step 3: P(V)=0.3795, P(C)=0.6205


In [243]:
# Compute the steady-state distribution
eigenvalues, eigenvectors = np.linalg.eig(T.T)

print("Eigenvalues:", eigenvalues)
print("Eigenvectors:\n", eigenvectors)

# Find the eigenvector corresponding to eigenvalue 1
eigenvector_to_ev_1 = eigenvectors[:, 1]

# Normalize to get probabilities
stationary_dist = eigenvector_to_ev_1 / eigenvector_to_ev_1.sum()

print(f"\nStationary distribution: {stationary_dist}")
print(f"P(V) = {stationary_dist[0]:.4f}, P(C) = {stationary_dist[1]:.4f}")

Eigenvalues: [-0.36600563  1.        ]
Eigenvectors:
 [[-0.70710678 -0.53090001]
 [ 0.70710678 -0.84743447]]

Stationary distribution: [0.38517502 0.61482498]
P(V) = 0.3852, P(C) = 0.6148


:::{.callout-note}
## Numpy and Eigenvalues

You may have noticed that we called `np.linalg.eig` with the transpose of the transition matrix.
This is because `np.linalg.eig` computes the right eigenvectors of a matrix, i.e., it solves the equation:

$$
\mathbf{A} \mathbf{v} = \lambda \mathbf{v}
$$

where $\mathbf{v}$ is a column vector. Your $\mathbf{p}_t$ vector is a row vector, so to find
the correct eigenvector we need to transpose the matrix:

$$
\mathbf{v}^T \mathbf{A} = \lambda \mathbf{v}^T
$$

:::



The TextRank algorithm scores sentences based on the stationary distribution of a Markov chain. Instead of webpages we have sentences. There are no real transition probabilities between sentences, but we can use the cosine similarity between the sentence representations in the TF-IDF space as a proxy.

Let's implement it as an exercise.

- Compute the TF-IDF matrix of the sentences
- Compute the cosine similarity matrix
- Normalize the cosine similarity matrix to get the transition matrix
- Smooth the transition matrix
- Compute the stationary distribution
- Rank the sentences based on the stationary distribution


In [244]:
# We already have th TF-IDF matrix X from before

print("X shape:", X.shape)
print(X)

X shape: (11, 113)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.27769841 0.         0.        ]
 [0.23406501 0.23406501 0.         ... 0.         0.         0.20007025]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.27019104 ... 0.         0.         0.23094946]]


In [245]:
# Now we compute the cosine similarity matrix
# between each pair of sentences

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_matrix = cosine_similarity(X)

# print("Cosine similarity shape:", cosine_sim_matrix.shape)
# print("Cosine Similarity Matrix:\n", cosine_sim_matrix)

In [246]:
# This matrix is no transition matrix, though 
# as the rows do not sum to 1.

T = cosine_sim_matrix / cosine_sim_matrix.sum(axis=1, keepdims=True)
T.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

The PageRank algorithm applies smoothing to the transition matrix as in practice it is not possible for every page to link to every other page.

The smoothing is done by adding a damping factor $d$ to the transition matrix

$$
P = \alpha T + (1-\alpha) E, \quad \alpha \in [0, 1]
$$

where $T$ is the original transition matrix and $E$ is a matrix with all elements equal to $1/n$.

In [247]:
alpha = 0.1

E = np.ones_like(T) / T.shape[0]
P = alpha * T + (1 - alpha) * E

In [248]:
P.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [249]:
# Now we need to find the stationary distribution of P

evals, evecs = np.linalg.eig(P.T)
print("Eigenvalues:\n", evals)

Eigenvalues:
 [1.         0.1        0.048277   0.08351595 0.07913968 0.05467085
 0.05919873 0.06090568 0.06362465 0.07016773 0.06760339]


In [250]:
ev_stationary = evecs[:,0]
ev_stationary

array([0.3018511 , 0.30110402, 0.30234851, 0.30366103, 0.3004944 ,
       0.30047697, 0.30178986, 0.30069743, 0.30151006, 0.3012753 ,
       0.30140199])

In [251]:
stationary_dist = ev_stationary / ev_stationary.sum()
stationary_dist

array([0.09101192, 0.09078666, 0.09116189, 0.09155763, 0.09060286,
       0.0905976 , 0.09099345, 0.09066407, 0.09090909, 0.09083831,
       0.09087651])